# Проверка изоморфизма с помощью хеш-функций


## Введение

**Определение:** Два объекта называются *изоморфными*, если между ними существует взаимно-однозначное отображение, сохраняющее структуру объектов.

В теории графов два графа $G_1 = (V_1, E_1)$ и $G_2 = (V_2, E_2)$ называются изоморфными, если существует биекция $f: V_1 \rightarrow V_2$ такая, что для любых вершин $u, v \in V_1$ выполняется условие: $(u, v) \in E_1$ тогда и только тогда, когда $(f(u), f(v)) \in E_2$.

Проблема изоморфизма графов имеет важное теоретическое и практическое значение:

- В теоретической информатике она относится к классу NP, но не доказано, что она NP-полна
- В химической информатике используется для определения эквивалентности молекулярных структур
- В компьютерном зрении применяется для сопоставления образов и распознавания объектов
- В компиляторах используется для оптимизации кода через распознавание эквивалентных подвыражений

Проверка изоморфизма объектов является вычислительно сложной задачей. Наивный подход требует перебора всех возможных отображений, что имеет факториальную сложность. Однако существуют эффективные эвристические методы, основанные на хеш-функциях, которые позволяют быстро отсеивать заведомо неизоморфные объекты и в некоторых случаях доказывать изоморфизм.


## Инварианты графов и хеш-функции

**Определение:** *Инвариант графа* — это функция, которая сопоставляет графу некоторое значение, одинаковое для всех изоморфных графов.

Формально, если $I$ — инвариант, а $G_1$ и $G_2$ — изоморфные графы, то $I(G_1) = I(G_2)$.

Важно отметить, что обратное утверждение не всегда верно: из равенства $I(G_1) = I(G_2)$ не следует, что $G_1$ и $G_2$ изоморфны. Такие инварианты называются *неполными*.

**Теорема:** Не существует полиномиально вычислимого полного инварианта графов, если P ≠ NP.

**Доказательство:**
Предположим, что существует полиномиально вычислимый полный инвариант $I$. Тогда для проверки изоморфизма графов $G_1$ и $G_2$ достаточно вычислить $I(G_1)$ и $I(G_2)$ и сравнить результаты. Если $I(G_1) = I(G_2)$, то $G_1$ и $G_2$ изоморфны, иначе — нет.

Поскольку $I$ вычисляется за полиномиальное время, вся процедура проверки изоморфизма также выполняется за полиномиальное время. Это означает, что задача изоморфизма графов принадлежит классу P.

Однако задача изоморфизма графов считается кандидатом на принадлежность к классу NP-промежуточных задач (задач, которые находятся в NP, но не являются ни P, ни NP-полными). Если P ≠ NP и задача изоморфизма графов не принадлежит P, то мы получаем противоречие.

Следовательно, если P ≠ NP, то не существует полиномиально вычислимого полного инварианта графов. $\blacksquare$

Несмотря на отсутствие полных инвариантов, неполные инварианты могут быть очень полезны для быстрой фильтрации заведомо неизоморфных графов. Хеш-функции предоставляют эффективный способ вычисления таких инвариантов.


In [ ]:
# Примеры простых инвариантов графа
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

def graph_invariants(G):
    """
    Вычисляет набор простых инвариантов графа
    
    Параметры:
    G (networkx.Graph): Граф
    
    Возвращает:
    dict: Словарь инвариантов
    """
    invariants = {}
    
    # Количество вершин и рёбер
    invariants['num_vertices'] = G.number_of_nodes()
    invariants['num_edges'] = G.number_of_edges()
    
    # Степени вершин (отсортированные)
    degrees = sorted([d for n, d in G.degree()])
    invariants['degree_sequence'] = degrees
    
    # Спектр графа (собственные значения матрицы смежности, отсортированные)
    adj_matrix = nx.to_numpy_array(G)
    eigenvalues = sorted(np.linalg.eigvals(adj_matrix).real)
    invariants['spectrum'] = [round(ev, 6) for ev in eigenvalues]  # Округляем для устранения численных погрешностей
    
    # Количество треугольников
    triangles = sum(nx.triangles(G).values()) // 3
    invariants['triangles'] = triangles
    
    # Диаметр графа (максимальное расстояние между любыми двумя вершинами)
    if nx.is_connected(G):
        invariants['diameter'] = nx.diameter(G)
    else:
        invariants['diameter'] = float('inf')
    
    return invariants

# Создаём два изоморфных графа с разной нумерацией вершин
G1 = nx.Graph()
G1.add_edges_from([(0, 1), (0, 2), (1, 2), (1, 3), (2, 3), (3, 4)])

G2 = nx.Graph()
G2.add_edges_from([(0, 1), (0, 3), (1, 2), (1, 3), (2, 4), (3, 4)])

# Создаём неизоморфный граф
G3 = nx.Graph()
G3.add_edges_from([(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3), (3, 4)])

# Вычисляем инварианты
inv1 = graph_invariants(G1)
inv2 = graph_invariants(G2)
inv3 = graph_invariants(G3)

# Визуализируем графы
plt.figure(figsize=(15, 5))

plt.subplot(131)
nx.draw(G1, with_labels=True, node_color='lightblue', node_size=500, font_weight='bold')
plt.title("Граф G1")

plt.subplot(132)
nx.draw(G2, with_labels=True, node_color='lightblue', node_size=500, font_weight='bold')
plt.title("Граф G2 (изоморфен G1)")

plt.subplot(133)
nx.draw(G3, with_labels=True, node_color='lightblue', node_size=500, font_weight='bold')
plt.title("Граф G3 (не изоморфен G1 и G2)")

plt.tight_layout()
plt.show()

# Выводим инварианты
print("Инварианты графа G1:")
for key, value in inv1.items():
    print(f"{key}: {value}")

print("\nИнварианты графа G2:")
for key, value in inv2.items():
    print(f"{key}: {value}")

print("\nИнварианты графа G3:")
for key, value in inv3.items():
    print(f"{key}: {value}")

print("\nG1 и G2 имеют одинаковые инварианты:", inv1 == inv2)
print("G1 и G3 имеют одинаковые инварианты:", inv1 == inv3)


## Канонические формы и хеш-функции для графов

**Определение:** *Каноническая форма* графа — это представление графа, которое однозначно определяется его структурой и не зависит от нумерации вершин.

Формально, каноническая форма — это функция $C$, которая сопоставляет каждому графу $G$ некоторый граф $C(G)$ такой, что:
1. $C(G)$ изоморфен $G$
2. Если $G_1$ и $G_2$ изоморфны, то $C(G_1) = C(G_2)$
3. Если $G_1$ и $G_2$ не изоморфны, то $C(G_1) \neq C(G_2)$

Каноническая форма является полным инвариантом, поскольку она однозначно определяет класс изоморфизма графа. Однако вычисление канонической формы в общем случае также является сложной задачей.

### Алгоритм Вейсфейлера-Лемана

Одним из эффективных методов вычисления хеш-значений графов является алгоритм Вейсфейлера-Лемана (Weisfeiler-Lehman), который итеративно уточняет метки вершин на основе их окружения.

**Теорема:** Если графы $G_1$ и $G_2$ не изоморфны, то с вероятностью не менее $1 - \frac{1}{2^k}$ алгоритм Вейсфейлера-Лемана с $k$ итерациями даст различные хеш-значения для этих графов.

**Доказательство (схема):**
Алгоритм Вейсфейлера-Лемана основан на итеративном уточнении меток вершин. На каждой итерации метка вершины обновляется на основе меток её соседей. Если после некоторого числа итераций мультимножества меток вершин двух графов различаются, то графы не изоморфны.

Можно показать, что если графы не изоморфны, то с вероятностью не менее $1 - \frac{1}{2^k}$ их мультимножества меток будут различаться после $k$ итераций, при условии использования хорошей хеш-функции для объединения меток.

Полное доказательство требует рассмотрения структурных свойств графов и свойств хеш-функций, что выходит за рамки данного изложения. $\blacksquare$


In [ ]:
def weisfeiler_lehman_hash(G, iterations=3):
    """
    Вычисляет хеш графа с помощью алгоритма Вейсфейлера-Лемана
    
    Параметры:
    G (networkx.Graph): Граф
    iterations (int): Количество итераций алгоритма
    
    Возвращает:
    str: Хеш-значение графа
    """
    # Инициализация: присваиваем всем вершинам одинаковую метку
    labels = {node: '1' for node in G.nodes()}
    
    # Итеративное уточнение меток
    for _ in range(iterations):
        new_labels = {}
        for node in G.nodes():
            # Собираем метки соседей
            neighbor_labels = [labels[neighbor] for neighbor in G.neighbors(node)]
            neighbor_labels.sort()
            
            # Формируем новую метку как комбинацию текущей метки и меток соседей
            new_label = labels[node] + '_' + '_'.join(neighbor_labels)
            new_labels[node] = new_label
        
        # Заменяем метки на их хеш-значения для сокращения длины
        label_dict = {label: str(i) for i, label in enumerate(sorted(set(new_labels.values())))}
        labels = {node: label_dict[new_labels[node]] for node in G.nodes()}
    
    # Формируем итоговый хеш как отсортированный список меток
    return '_'.join(sorted(labels.values()))

# Применяем алгоритм Вейсфейлера-Лемана к нашим графам
wl_hash1 = weisfeiler_lehman_hash(G1)
wl_hash2 = weisfeiler_lehman_hash(G2)
wl_hash3 = weisfeiler_lehman_hash(G3)

print("Хеш Вейсфейлера-Лемана для G1:", wl_hash1)
print("Хеш Вейсфейлера-Лемана для G2:", wl_hash2)
print("Хеш Вейсфейлера-Лемана для G3:", wl_hash3)
print("\nG1 и G2 имеют одинаковые хеши:", wl_hash1 == wl_hash2)
print("G1 и G3 имеют одинаковые хеши:", wl_hash1 == wl_hash3)


## Изоморфизм строк и деревьев

### Изоморфизм строк

**Определение:** Две строки $s$ и $t$ называются *изоморфными*, если существует биекция между символами алфавита, которая переводит $s$ в $t$.

Например, строки "abba" и "cddc" изоморфны, так как можно установить соответствие $a \leftrightarrow c$, $b \leftrightarrow d$. А строки "abba" и "cddd" не изоморфны, так как в первой строке два различных символа, а во второй — только один.

**Теорема:** Проверка изоморфизма строк может быть выполнена за линейное время $O(n)$, где $n$ — длина строк.

**Доказательство:**
Для проверки изоморфизма строк $s$ и $t$ длины $n$ можно использовать следующий алгоритм:

1. Если длины строк различаются, то строки не изоморфны.
2. Создаём два отображения: $map_{s \to t}$ и $map_{t \to s}$.
3. Проходим по строкам одновременно:
   - Если символ $s[i]$ уже отображается в $map_{s \to t}$, проверяем, что $map_{s \to t}[s[i]] = t[i]$.
   - Если символ $t[i]$ уже отображается в $map_{t \to s}$, проверяем, что $map_{t \to s}[t[i]] = s[i]$.
   - Иначе добавляем отображения $s[i] \to t[i]$ в $map_{s \to t}$ и $t[i] \to s[i]$ в $map_{t \to s}$.
4. Если на каком-то шаге проверка не выполняется, строки не изоморфны.

Алгоритм выполняет один проход по строкам, используя константное время на каждом шаге, поэтому его временная сложность — $O(n)$. $\blacksquare$


In [ ]:
def are_strings_isomorphic(s, t):
    """
    Проверяет, являются ли две строки изоморфными
    
    Параметры:
    s (str): Первая строка
    t (str): Вторая строка
    
    Возвращает:
    bool: True, если строки изоморфны, иначе False
    """
    if len(s) != len(t):
        return False
    
    s_to_t = {}
    t_to_s = {}
    
    for i in range(len(s)):
        if s[i] in s_to_t:
            if s_to_t[s[i]] != t[i]:
                return False
        else:
            s_to_t[s[i]] = t[i]
        
        if t[i] in t_to_s:
            if t_to_s[t[i]] != s[i]:
                return False
        else:
            t_to_s[t[i]] = s[i]
    
    return True

# Примеры
test_cases = [
    ("abba", "cddc"),  # Изоморфны
    ("abba", "cddd"),  # Не изоморфны
    ("aabb", "ccdd"),  # Изоморфны
    ("abcd", "abcd"),  # Изоморфны
    ("abcd", "aaaa"),  # Не изоморфны
]

for s, t in test_cases:
    print(f"'{s}' и '{t}' {'изоморфны' if are_strings_isomorphic(s, t) else 'не изоморфны'}")


### Изоморфизм деревьев

**Определение:** Два дерева $T_1$ и $T_2$ называются *изоморфными*, если существует биекция между их вершинами, сохраняющая отношение "родитель-потомок".

Проверка изоморфизма деревьев может быть выполнена эффективно с помощью хеш-функций, основанных на структуре дерева.

**Теорема:** Существует алгоритм проверки изоморфизма деревьев со сложностью $O(n)$, где $n$ — количество вершин в деревьях.

**Доказательство (схема):**
Алгоритм основан на вычислении хеш-значений для каждой вершины дерева, начиная с листьев и продвигаясь к корню. Хеш-значение вершины зависит от хеш-значений её потомков.

1. Для листьев хеш-значение устанавливается равным некоторой константе.
2. Для внутренних вершин хеш-значение вычисляется как хеш от мультимножества хеш-значений потомков.
3. Два дерева изоморфны тогда и только тогда, когда хеш-значения их корней совпадают.

Поскольку алгоритм посещает каждую вершину ровно один раз и выполняет константное число операций для каждой вершины, его временная сложность — $O(n)$. $\blacksquare$


In [ ]:
def tree_hash(root):
    """
    Вычисляет хеш-значение дерева с помощью рекурсивного алгоритма
    
    Параметры:
    root: Корень дерева (узел с атрибутами value и children)
    
    Возвращает:
    int: Хеш-значение дерева
    """
    if not root:
        return 0
    
    # Если у узла нет потомков, возвращаем хеш его значения
    if not root.children:
        return hash(root.value)
    
    # Вычисляем хеши потомков
    child_hashes = [tree_hash(child) for child in root.children]
    
    # Сортируем хеши потомков для обеспечения инвариантности относительно порядка
    child_hashes.sort()
    
    # Вычисляем хеш узла на основе его значения и хешей потомков
    return hash((root.value, tuple(child_hashes)))

# Определяем класс для узла дерева
class TreeNode:
    def __init__(self, value):
        self.value = value
        self.children = []
    
    def add_child(self, child):
        self.children.append(child)
        return child

# Создаём два изоморфных дерева с разной структурой
# Дерево 1:
#      A
#     / \
#    B   C
#   / \
#  D   E

tree1 = TreeNode("A")
node_b = tree1.add_child(TreeNode("B"))
tree1.add_child(TreeNode("C"))
node_b.add_child(TreeNode("D"))
node_b.add_child(TreeNode("E"))

# Дерево 2:
#      X
#     / \
#    Y   Z
#   / \
#  W   V

tree2 = TreeNode("X")
node_y = tree2.add_child(TreeNode("Y"))
tree2.add_child(TreeNode("Z"))
node_y.add_child(TreeNode("W"))
node_y.add_child(TreeNode("V"))

# Дерево 3 (не изоморфно деревьям 1 и 2):
#      A
#     / \
#    B   C
#       / \
#      D   E

tree3 = TreeNode("A")
tree3.add_child(TreeNode("B"))
node_c = tree3.add_child(TreeNode("C"))
node_c.add_child(TreeNode("D"))
node_c.add_child(TreeNode("E"))

# Вычисляем хеши деревьев
hash1 = tree_hash(tree1)
hash2 = tree_hash(tree2)
hash3 = tree_hash(tree3)

print("Хеш дерева 1:", hash1)
print("Хеш дерева 2:", hash2)
print("Хеш дерева 3:", hash3)
print("\nДеревья 1 и 2 изоморфны:", hash1 == hash2)
print("Деревья 1 и 3 изоморфны:", hash1 == hash3)


## Алгоритм Мак-Кея (nauty)

Одним из наиболее эффективных практических алгоритмов для проверки изоморфизма графов является алгоритм Мак-Кея, реализованный в библиотеке nauty. Этот алгоритм вычисляет каноническую форму графа, что позволяет эффективно решать задачу изоморфизма.

**Теорема:** Алгоритм Мак-Кея корректно определяет, являются ли два графа изоморфными.

**Доказательство (схема):**
Алгоритм Мак-Кея основан на поиске канонической формы графа путём перебора возможных перестановок вершин с использованием эффективных эвристик для отсечения неперспективных ветвей поиска.

Ключевые идеи алгоритма:
1. Использование автоморфизмов графа для сокращения пространства поиска
2. Применение инвариантов для быстрого отсечения неизоморфных графов
3. Эффективная структура данных для представления графов и их перестановок

Полное доказательство корректности алгоритма Мак-Кея выходит за рамки данного изложения, но его эффективность подтверждена многочисленными практическими применениями. $\blacksquare$


In [ ]:
try:
    import pynauty

    def check_isomorphism_nauty(G1, G2):
        """
        Проверяет изоморфизм графов с помощью библиотеки pynauty
        
        Параметры:
        G1, G2 (networkx.Graph): Графы для проверки
        
        Возвращает:
        bool: True, если графы изоморфны, иначе False
        """
        # Преобразуем графы NetworkX в формат pynauty
        def nx_to_pynauty(G):
            n = G.number_of_nodes()
            edges = []
            for u, v in G.edges():
                edges.append((u, v))
            
            adjacency_dict = {i: set() for i in range(n)}
            for u, v in edges:
                adjacency_dict[u].add(v)
                adjacency_dict[v].add(u)
            
            return pynauty.Graph(n, adjacency_dict=adjacency_dict)
        
        g1 = nx_to_pynauty(G1)
        g2 = nx_to_pynauty(G2)
        
        # Вычисляем канонические формы
        canon1 = pynauty.canon_graph(g1)
        canon2 = pynauty.canon_graph(g2)
        
        # Сравниваем канонические формы
        return canon1 == canon2
    
    # Проверяем изоморфизм наших графов
    print("Проверка изоморфизма с помощью nauty:")
    print("G1 и G2 изоморфны:", check_isomorphism_nauty(G1, G2))
    print("G1 и G3 изоморфны:", check_isomorphism_nauty(G1, G3))
    
except ImportError:
    print("Библиотека pynauty не установлена. Используем встроенный алгоритм NetworkX.")
    
    # Используем встроенный алгоритм NetworkX для проверки изоморфизма
    print("Проверка изоморфизма с помощью NetworkX:")
    print("G1 и G2 изоморфны:", nx.is_isomorphic(G1, G2))
    print("G1 и G3 изоморфны:", nx.is_isomorphic(G1, G3))


## Практические применения

### Химическая информатика

В химической информатике изоморфизм графов используется для определения эквивалентности молекулярных структур. Молекулы часто представляются в виде графов, где вершины — атомы, а рёбра — химические связи.

**Теорема:** Две молекулы имеют одинаковые химические свойства, если их молекулярные графы изоморфны (при условии, что учитываются все релевантные характеристики атомов и связей).

**Доказательство (схема):**
Химические свойства молекулы определяются её структурой, то есть расположением атомов и связей между ними. Если две молекулы имеют изоморфные графы, то существует взаимно-однозначное соответствие между их атомами и связями, сохраняющее структуру.

При условии, что в графовом представлении учитываются все релевантные характеристики (типы атомов, типы связей и т.д.), изоморфизм графов гарантирует идентичность химических свойств молекул. $\blacksquare$

### Компьютерное зрение

В компьютерном зрении изоморфизм графов используется для сопоставления образов и распознавания объектов. Объекты могут быть представлены в виде графов, где вершины — характерные точки, а рёбра — отношения между ними.

**Теорема:** Если графы признаков двух изображений изоморфны, то с высокой вероятностью эти изображения содержат один и тот же объект (при условии, что графы признаков достаточно информативны).

**Доказательство (схема):**
Граф признаков изображения содержит информацию о структуре объекта. Если графы признаков двух изображений изоморфны, то существует взаимно-однозначное соответствие между характерными точками объектов, сохраняющее их структурные отношения.

При условии, что графы признаков достаточно информативны (учитывают геометрические, текстурные и другие характеристики), изоморфизм графов указывает на высокую вероятность того, что изображения содержат один и тот же объект. $\blacksquare$


In [ ]:
# Пример применения изоморфизма в химической информатике
try:
    from rdkit import Chem
    from rdkit.Chem import Draw
    
    # Создаём молекулы
    mol1 = Chem.MolFromSmiles("CC(=O)OC1=CC=CC=C1C(=O)O")  # Аспирин
    mol2 = Chem.MolFromSmiles("O=C(C)Oc1ccccc1C(=O)O")     # Аспирин (другая запись)
    mol3 = Chem.MolFromSmiles("CC(=O)OC1=CC=CC=C1")        # Фенилацетат (не аспирин)
    
    # Проверяем изоморфизм
    print("Химическая информатика:")
    print("Молекулы 1 и 2 изоморфны:", mol1.GetNumAtoms() == mol2.GetNumAtoms() and 
          Chem.MolToSmiles(mol1) == Chem.MolToSmiles(mol2))
    print("Молекулы 1 и 3 изоморфны:", mol1.GetNumAtoms() == mol3.GetNumAtoms() and 
          Chem.MolToSmiles(mol1) == Chem.MolToSmiles(mol3))
    
    # Визуализируем молекулы
    img = Draw.MolsToGridImage([mol1, mol2, mol3], 
                              molsPerRow=3, 
                              subImgSize=(200, 200), 
                              legends=["Аспирин 1", "Аспирин 2", "Фенилацетат"])
    img.save("molecules.png")
    print("Изображения молекул сохранены в файл molecules.png")
    
except ImportError:
    print("Библиотека RDKit не установлена. Пример химической информатики пропущен.")


## Заключение

Проверка изоморфизма с помощью хеш-функций представляет собой мощный инструмент для решения различных задач в информатике, химии, компьютерном зрении и других областях. Основные выводы:

1. **Теоретическая сложность:** Задача изоморфизма графов принадлежит классу NP, но не доказано, что она NP-полна. Для некоторых классов графов (деревья, планарные графы) существуют полиномиальные алгоритмы.

2. **Практические алгоритмы:** Алгоритмы, основанные на хеш-функциях (Вейсфейлера-Лемана) и канонических формах (Мак-Кея), позволяют эффективно решать задачу изоморфизма на практике.

3. **Инварианты:** Использование инвариантов графов позволяет быстро отсеивать заведомо неизоморфные графы, что существенно ускоряет процесс проверки.

4. **Применения:** Методы проверки изоморфизма находят широкое применение в различных областях, от химической информатики до компьютерного зрения и оптимизации компиляторов.

Несмотря на теоретическую сложность задачи, современные алгоритмы и хеш-функции позволяют эффективно решать задачу изоморфизма для большинства практических случаев.


## Библиография

1. Babai, L. (2016). Graph Isomorphism in Quasipolynomial Time. In Proceedings of the 48th Annual ACM SIGACT Symposium on Theory of Computing (STOC 2016).

2. McKay, B. D., & Piperno, A. (2014). Practical Graph Isomorphism, II. Journal of Symbolic Computation, 60, 94-112.

3. Weisfeiler, B., & Lehman, A. A. (1968). A Reduction of a Graph to a Canonical Form and an Algebra Arising During this Reduction. Nauchno-Technicheskaya Informatsia, Ser. 2, 9, 12-16.

4. Shervashidze, N., Schweitzer, P., van Leeuwen, E. J., Mehlhorn, K., & Borgwardt, K. M. (2011). Weisfeiler-Lehman Graph Kernels. Journal of Machine Learning Research, 12, 2539-2561.

5. Cordella, L. P., Foggia, P., Sansone, C., & Vento, M. (2004). A (Sub)Graph Isomorphism Algorithm for Matching Large Graphs. IEEE Transactions on Pattern Analysis and Machine Intelligence, 26(10), 1367-1372.

6. Ullmann, J. R. (1976). An Algorithm for Subgraph Isomorphism. Journal of the ACM, 23(1), 31-42.

7. Кормен Т., Лейзерсон Ч., Ривест Р., Штайн К. Алгоритмы: построение и анализ. — М.: Вильямс, 2013.

8. Харари Ф. Теория графов. — М.: Мир, 1973.

9. Алексеев В.Б., Таланов В.А. Графы. Модели вычислений. Структуры данных. — М.: Бином. Лаборатория знаний, 2012.

10. Емеличев В.А., Мельников О.И., Сарванов В.И., Тышкевич Р.И. Лекции по теории графов. — М.: Наука, 1990.